In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
from scipy.stats import expon

data = pd.read_csv(r'dados/movie_metadata.csv')
budgets = [0.1, 1, 10]

data.drop_duplicates()
for i in range(8, 21):
    print(str(i) + ": " + str(data.columns[i]))
# data.head()

8: gross
9: genres
10: actor_1_name
11: movie_title
12: num_voted_users
13: cast_total_facebook_likes
14: actor_3_name
15: facenumber_in_poster
16: plot_keywords
17: movie_imdb_link
18: num_user_for_reviews
19: language
20: country


# Consultas

In [2]:
# Consulta 1: Filme com a maior arrecadação
def consulta1(data):
    aux = data[['movie_title', 'gross'][:]].dropna()
    return aux.loc[aux.gross.idxmax()]

# Consulta 2: Filme com maior arrecadação para cada idioma
def consulta2(data):
    aux = data[['movie_title', 'language', 'gross'][:]].dropna()
    return aux.loc[aux.groupby('language').gross.idxmax()]

# Consulta 3: Top 3 países com a maior quantidade de filmes
def consulta3(data):
    result = []
    aux = data[['country', 'movie_title'][:]].dropna()
    country_list = aux.groupby('country')['movie_title'].nunique().to_frame()
    country_list['country'] = country_list.index # o índice se perde ao agrupar
    country_list.reset_index(drop=True, inplace=True)
    for i in range(3):
        result.append(country_list.loc[country_list.movie_title.idxmax()])
        country_list = country_list.drop(country_list.movie_title.idxmax())
    return pd.DataFrame(result)

# Funções Score

In [3]:
def score1(item, out):
    score = 0
    if(not np.isnan(item.gross) and item.movie_title != 'None'):
        if(item.movie_title == out):
            score = item.gross
    return score

def score2(item, out):
    score = 0
    if(item.movie_title in out and not np.isnan(item.gross)):
        score = item.gross
    return score

def score3(data, item, out):
    score = 0
    total_countries = data.country.array
    country_list = {}
    for i in range(0, len(total_countries)):
        if total_countries[i] not in country_list:
            country_list[total_countries[i]] = 0
            
    for i in range(0, len(data)):
        if type(data.iloc[i].country != float):
            country_list[data.iloc[i].country] += 1
    if item.country in out:
        score = country_list[item.country]
    return score

# Funções de sensibilidade

In [ ]:
def sense_q1(data):
    mat = []
    sense = 0
    results = data[['movie_title'][:]]
    for i in range(0, len(results)):
        mat.append(score1(data.iloc[i], results.iloc[i].movie_title))
        
    for i in range(0, len(mat) - 1):
        sense = max([sense, abs(mat[i] - mat[i + 1])])
    return sense

def sense_q2(data):
    mat = []
    sense = 0
    results = data[['movie_title', 'language', 'gross'][:]]
    for i in range(0, len(results)):
        mat.append(score2(results.iloc[i], data.iloc[i].movie_title))
    
    for i in range(0, len(mat) - 1):
        sense = max([sense, abs(mat[i] - mat[i + 1])])
    return sense

def sense_q3(data):
    sense = 0
    country_list = []
    mat = []
    for i in range(0, len(data)):
        if(data.iloc[i].country not in country_list and data.iloc[i].country != 'None'):
            country_list.append(data.iloc[i].country)
    results = list(combinations(country_list, 3))
    
    for r in range(0, len(results)):
        mat.append(score3(data, data.iloc[r], results[r]))

    for i in range(0, len(mat) - 1):
        sense = max([sense, abs(mat[i] - mat[i + 1])])
    return sense

sense_q3(data)

# Mecanismo Exponencial

# Randomização de consultas

# Execução

In [ ]:
# TODO